# Diabetes-Vorhersage mit Support Vector Machine (SVM)

### 1. Aufgabenstellung

In diesem Kapitel wirst du ein vollständiges Machine-Learning-Projekt umsetzen. Ziel ist es, ein System zu entwickeln, das anhand medizinischer Parameter vorhersagen kann, ob eine Person an Diabetes leidet oder nicht.

Dafür verwenden wir:

Einen klassischen Supervised Learning Algorithmus: den Support Vector Machine (SVM) Classifier.

Den Pima-Indians-Diabetes-Datensatz, welcher medizinische Informationen über weibliche Patienten enthält.

**Zielsetzung**:

Trainiere ein SVM-Modell, das zwischen "diabetisch" und "nicht-diabetisch" unterscheiden kann.

Finde die Genauigkeit des Modells auf Trainings- und Testdaten.

Erstelle ein Vorhersagesystem, das neue Patientendaten klassifiziert.

## 2. Hinweise & Tipps

Der Datensatz enthält 8 Merkmale wie z. B. Glukosewert, BMI, Blutdruck usw. Das Label "Outcome" gibt an, ob die Person Diabetes hat (1) oder nicht (0).

Standardisierung der Daten ist wichtig, da die Skalen der Merkmale unterschiedlich sind.

Nutze Train-Test-Split, um Überanpassung zu vermeiden.

Achte bei der Modellbewertung sowohl auf das Training als auch auf das Testing.

Verwende Numpy und Pandas zur Datenmanipulation und Scikit-Learn für ML-Komponenten.

## 3. Lösungsschritte inkl. Erklärungen



### Abhängigkeiten importieren

Bevor wir überhaupt mit dem eigentlichen Machine-Learning-Modell anfangen können, müssen wir ein paar wichtige Bibliotheken in Python importieren. Diese stellen uns Funktionen bereit, die wir für die Datenverarbeitung, das Modelltraining und die Auswertung brauchen.

Wir verwenden numpy für mathematische Operationen, pandas zum Laden und Bearbeiten der Daten. StandardScaler brauchen wir später zur Standardisierung der Werte, train_test_split für die Aufteilung der Daten, svm für unser Support Vector Machine Modell und accuracy_score, um die Genauigkeit des Modells zu messen.

In [50]:
import numpy as np
import pandas as pd  # numpy und pandas für Datenverarbeitung,
from sklearn.preprocessing import StandardScaler # StandardScaler zur Standardisierung der Daten,
from sklearn.model_selection import train_test_split # train_test_split zur Aufteilung in Trainings- und Testdaten,
from sklearn import svm # svm für den Support Vector Machine Algorithmus,
from sklearn.metrics import accuracy_score # accuracy_score zur Bewertung der Modellgenauigkeit

### Datensatz laden und überblicken

Jetzt, wo die Werkzeuge bereit sind, können wir den Datensatz laden. Wir verwenden hier den Pima-Indians-Diabetes-Datensatz. Der enthält medizinische Werte von Patientinnen – also z. B. Glukose, Blutdruck, BMI usw.
Dadurch bekommen wir einen Eindruck über Struktur und Inhalt des Datensatzes (768 Zeilen, 9 Spalten).

In [51]:
# Laden des Diabetes-Datensatzes aus einer CSV-Datei in ein pandas DataFrame
diabetes_dataset = pd.read_csv('../../Data/Project 2/diabetes.csv') 

Wir lesen die Datei ein und speichern sie in einer Variable namens diabetes_dataset, die jetzt ein sogenannter DataFrame ist – das ist im Grunde eine Tabelle mit Zeilen und Spalten
Bevor wir mit der Analyse weitermachen, schauen wir uns die Daten an. Das ist wie ein erster Blick auf eine Excel-Tabelle – wir wollen einfach ein Gefühl für den Aufbau bekommen.

In [52]:
# Anzeige der ersten 5 Zeilen des Datensatzes zur ersten Orientierung
diabetes_dataset.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Mit head() sehen wir die ersten 5 Zeilen, describe() gibt uns statistische Infos wie Mittelwert, Standardabweichung oder Minimum und Maximum. Das hilft, ein Gefühl für die Wertebereiche zu bekommen.“

In [53]:
# Ausgabe statistischer Kennzahlen wie Mittelwert, Standardabweichung, Min/Max usw.
diabetes_dataset.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


### Labels analysieren

Wir wollen auch wissen, wie viele Personen im Datensatz diabetisch sind und wie viele nicht. Das ist wichtig, um zu sehen, ob die Daten ausgeglichen sind.
Das Label Outcome ist entweder 0 (nicht-diabetisch) oder 1 (diabetisch). Die Verteilung zeigt, ob unser Modell später genug Beispiele von beiden Klassen sieht.
Gibt uns Informationen über das Verhältnis von diabetischen (1) und nicht-diabetischen (0) Patienten.

In [54]:
diabetes_dataset['Outcome'].value_counts() # zählt, wie oft die Klassen 0 (nicht-diabetisch) und 1 (diabetisch) im Label 'Outcome' vorkommen

Outcome
0    500
1    268
Name: count, dtype: int64

### Gruppieren und Mittelwerte vergleichen

Jetzt schauen wir uns an, wie sich die Werte zwischen diabetischen und nicht-diabetischen Personen unterscheiden. Vielleicht sehen wir z. B., dass Diabetiker im Schnitt höhere Glukosewerte haben



### Merkmale (X) und Label (y) trennen

In [55]:
# Berechnet den Durchschnitt jeder Spalte getrennt nach dem Wert von 'Outcome' (0 oder 1)

diabetes_dataset.groupby('Outcome').mean()  # → Zeigt z. B., ob diabetische Patienten im Durchschnitt höhere Glukosewerte haben


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.298000,109.980000,68.184000,19.664000,68.792000,30.304200,0.429734,31.190000
1,4.865672,141.257463,70.824627,22.164179,100.335821,35.142537,0.550500,37.067164


Das ist spannend, denn solche Unterschiede kann unser Modell später lernen und nutzen, um Vorhersagen zu treffen.
Zeigt uns signifikante Unterschiede zwischen den beiden Gruppen (z. B. im Glukosewert).

### Standardisierung der Merkmale

Als Nächstes trennen wir die Daten in zwei Bereiche: Die Eingabedaten (X) und das Ziel bzw. Label (Y). Das Ziel ist, dass das Modell aus X vorhersagen soll, was in Y steht

In [56]:
# Trennen der Eingabedaten (X) von den Zielwerten/Labels (Y)

# X enthält alle Spalten außer 'Outcome',
X = diabetes_dataset.drop(columns = 'Outcome', axis=1)

# Y enthält nur die 'Outcome'-Spalte
Y = diabetes_dataset['Outcome']

In [57]:
# Ausgabe der Eingabedaten (X) -> Man sieht, dass X alle Spalten enthält 
print(X)

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  
0                       0.627   50  
1                       0.351   31  


In [58]:
print(Y) # Ausgabe der Zielwerte (Y) zur Kontrolle. Man sieht, dass Y nur Wete aus eine Spalte liefert 

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64


### Standardisierung der Merkmale

Jetzt kommt ein ganz wichtiger Schritt: die Standardisierung. Manche Werte wie Glukose oder Insulin haben viel größere Zahlen als z. B. die Anzahl der Schwangerschaften. Das kann ein Problem sein.

In [59]:
# Erstellen eines StandardScaler-Objekts zur späteren Standardisierung der Merkmale

scaler = StandardScaler()



In [60]:
# "Lernen" der Mittelwerte und Standardabweichungen aus den Trainingsdaten X
# (Diese werden für die spätere Transformation verwendet)

scaler.fit(X)

,copy,True
,with_mean,True
,with_std,True


In [61]:
#  Anwenden der Standardisierung: Skalieren der Daten auf Mittelwert = 0 und Standardabweichung = 1

standardized_data = scaler.transform(X)

In [62]:
#  Ausgabe der standardisierten Daten zur Überprüfung – alle Werte liegen jetzt im ähnlichen Bereich

print(standardized_data)

[[ 0.63994726  0.84832379  0.14964075 ...  0.20401277  0.46849198
   1.4259954 ]
 [-0.84488505 -1.12339636 -0.16054575 ... -0.68442195 -0.36506078
  -0.19067191]
 [ 1.23388019  1.94372388 -0.26394125 ... -1.10325546  0.60439732
  -0.10558415]
 ...
 [ 0.3429808   0.00330087  0.14964075 ... -0.73518964 -0.68519336
  -0.27575966]
 [-0.84488505  0.1597866  -0.47073225 ... -0.24020459 -0.37110101
   1.17073215]
 [-0.84488505 -0.8730192   0.04624525 ... -0.20212881 -0.47378505
  -0.87137393]]


Wir bringen deshalb alle Merkmale auf die gleiche Skala – mit Mittelwert 0 und Standardabweichung 1. So lernt unser Modell effizienter

In [63]:
#  Überschreiben von X mit den standardisierten Daten
X = standardized_data

# Y bleibt das Ziel (Label) – enthält weiterhin die Outcome-Werte (0 oder 1)
Y = diabetes_dataset['Outcome']

### Aufteilung in Trainings- und Testdaten

Wir teilen die Daten auf: einen Teil fürs Training, einen Teil zum Testen. So prüfen wir später, wie gut das Modell auf neuen, unbekannten Daten funktioniert.
80% Training, 20% Test. stratify=y sorgt für gleiche Verteilung der Labels in beiden Sets.

In [64]:
# Aufteilen der Daten in Trainings- und Testsets
# 80 % Training, 20 % Test – mit gleichmäßiger Verteilung der Klassen dank `stratify=Y`
# `random_state=2` sorgt für Reproduzierbarkeit der Aufteilung

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, stratify=Y, random_state=2)

20 % der Daten verwenden wir fürs Testen. Dank stratify=Y bleibt das Verhältnis von 0 und 1 erhalten

In [65]:
#  Ausgabe der Formen (Anzahl Zeilen & Spalten) der Gesamtdaten, Trainingsdaten und Testdaten


print(X.shape, X_train.shape, X_test.shape) # → Kontrolle, ob die Aufteilung korrekt erfolgt ist

(768, 8) (614, 8) (154, 8)


####  Modell erstellen und trainieren

Jetzt erstellen wir unser Support Vector Machine Modell und trainieren es mit den Trainingsdaten

In [66]:
#  Erstellen eines SVM-Klassifikators mit linearem Kernel

classifier = svm.SVC(kernel='linear') # → geeignet für Daten, die (nahezu) linear trennbar sind

In [67]:
#  Training des SVM-Klassifikators mit den Trainingsdaten

# Das Modell "lernt", wie es zwischen diabetisch (1) und nicht-diabetisch (0) unterscheiden kann
classifier.fit(X_train, Y_train)

,C,1.0
,kernel,'linear'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


Der linear Kernel ist gut geeignet, wenn sich die Klassen mit einer geraden Linie trennen lassen – was bei diesen Daten oft der Fall ist.

#### Genauigkeit prüfen

Wir prüfen jetzt, wie gut das Modell gelernt hat – sowohl auf den Trainingsdaten als auch auf den Testdaten.

In [68]:
# Vorhersagen auf den Trainingsdaten treffen
X_train_prediction = classifier.predict(X_train)


#  Berechnung der Genauigkeit auf den Trainingsdaten

training_data_accuracy = accuracy_score(X_train_prediction, Y_train) # → Misst, wie viele Vorhersagen auf dem Trainingsset korrekt waren

In [69]:
#  Ausgabe der Genauigkeit des Modells auf den Trainingsdaten

print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.7866449511400652


In [70]:

#  Vorhersagen auf den Testdaten treffen
X_test_prediction = classifier.predict(X_test)

#  Berechnung der Genauigkeit auf den Testdaten

test_data_accuracy = accuracy_score(X_test_prediction, Y_test) # → Zeigt, wie gut das Modell auf unbekannten (nicht gesehenen) Daten performt

In [71]:
print('Accuracy score of the test data : ', test_data_accuracy) # Ausgabe der Genauigkeit des Modells auf den Testdaten

Accuracy score of the test data :  0.7727272727272727


Die Werte zeigen, ob das Modell gut generalisiert. Wenn die Testgenauigkeit viel schlechter ist als die Trainingsgenauigkeit, könnte es überangepasst sein.

### Vorhersagesystem bauen


Jetzt kommt der spannende Teil: Wir testen das Modell mit echten neuen Werten. Damit können wir sehen, ob es wirklich funktioniert

In [72]:
#  Beispielinput eines neuen Patienten (alle 8 medizinischen Merkmale)
input_data = (5, 166, 72, 19, 175, 25.8, 0.587, 51)

#  Umwandlung in ein NumPy-Array (notwendig für weitere Verarbeitung)
input_data_as_numpy_array = np.asarray(input_data)

#  Reshape: Umformen in 2D-Array, da das Modell Eingaben zeilenweise erwartet
input_data_reshaped = input_data_as_numpy_array.reshape(1, -1)

#  Standardisieren der Eingabedaten mit dem zuvor gelernten StandardScaler
std_data = scaler.transform(input_data_reshaped)
print(std_data)  # Zeigt die skalierten Werte

#  Vorhersage mit dem trainierten Modell
prediction = classifier.predict(std_data)
print(prediction)  # Gibt 0 (nicht-diabetisch) oder 1 (diabetisch) zurück

#  Interpretation des Ergebnisses
if prediction[0] == 0:
    print('The person is not diabetic')  # Modell sagt: kein Diabetes
else:
    print('The person is diabetic')      # Modell sagt: Diabetes

[[ 0.3429808   1.41167241  0.14964075 -0.09637905  0.82661621 -0.78595734
   0.34768723  1.51108316]]
[1]
The person is diabetic


e:\Data-Plus\MLL\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


So kannst du später auch mit echten Nutzereingaben arbeiten, zum Beispiel über eine Benutzeroberfläche.“
